In [1]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
import sqlalchemy as db
import pandas as pd

# from numpy import genfromtxt
from time import time
from datetime import datetime
from sqlalchemy import Column, Integer, Float, Date, String, VARCHAR
from sqlalchemy.ext.declarative import declarative_base

from sqlalchemy.orm import sessionmaker
import csv

In [2]:
params = {
    'host': '54.187.186.58',
    'user': 'ubuntu',
    'port': 5432
}

In [3]:
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/baseball'
engine = create_engine(connection_string)
connection = connect(**params, dbname='baseball')
# meta = MetaData()
cursor = connection.cursor()
meta = db.MetaData()
meta.create_all(engine)
engine.table_names()

['allstarfull', 'salaries', 'schools', 'college_playing', 'schools_playing']

## 1. What was the total spent on salaries by each team, each year?

In [34]:
query = '''
SELECT *
FROM salaries;
'''
salaries = pd.read_sql_query(query, connection)
salaries

,yearid,teamid,lgid,playerid,salary
0,1985,BAL,AL,murraed02,1472819.0
1,1985,BAL,AL,lynnfr01,1090000.0
2,1985,BAL,AL,ripkeca01,800000.0
3,1985,BAL,AL,lacyle01,725000.0
4,1985,BAL,AL,flanami01,641667.0
...,...,...,...,...,...
23951,2013,WAS,NL,matthry01,504500.0
23952,2013,WAS,NL,lombast02,501250.0
23953,2013,WAS,NL,ramoswi01,501250.0
23954,2013,WAS,NL,rodrihe03,501000.0


In [35]:
salaries.groupby(['teamid', 'yearid'])['salary'].sum()

teamid  yearid
ANA     1997       31135472.0
        1998       41281000.0
        1999       55388166.0
        2000       51464167.0
        2001       47535167.0
                     ...     
WAS     2009       59928000.0
        2010       61400000.0
        2011       63856928.0
        2012       80855143.0
        2013      113703270.0
Name: salary, Length: 828, dtype: float64

In [39]:
query = '''
SELECT teamid, yearid, SUM(salary) AS total_salary
FROM salaries
GROUP BY teamid, yearid;
'''
total_salary = pd.read_sql_query(query, connection)
total_salary

,teamid,yearid,total_salary
0,LAN,1987,13675403.0
1,NYA,1992,37543334.0
2,BAL,2013,84393333.0
3,TOR,1992,44788666.0
4,MIN,2004,53585000.0
...,...,...,...
823,TEX,2013,112522600.0
824,MON,1993,18899333.0
825,PIT,1995,18355345.0
826,PIT,1992,33944167.0


## 2. What is the first and last year played for each player?

In [56]:
Fielding = '/Users/greenapple/Downloads/baseballdatabank-2019.2/core/Fielding.csv'
Fielding_df = pd.read_csv(Fielding)

In [41]:
query = '''
SELECT playerid, MIN(yearid) AS first_year, MAX(yearid) AS last_year
FROM salaries
GROUP BY playerid;
'''
player_1_and_last_years = pd.read_sql_query(query, connection)
player_1_and_last_years

,playerid,first_year,last_year
0,saitota01,2007,2012
1,escaled01,2013,2013
2,ramirju01,2001,2003
3,suzukma01,1999,2001
4,meyerjo01,1988,1989
...,...,...,...
4663,halldr01,1987,1990
4664,joynewa01,1986,2001
4665,dolisra01,2012,2012
4666,gladdda01,1985,1993


## 3. Who has played the most all star games?

In [17]:
# Not clear whether 'gamenum' or 'gameid' should be used
query = '''
SELECT playerid, COUNT(gameid) AS game_num
FROM allstarfull
GROUP BY playerid
ORDER BY game_num DESC;
'''
game_num_by_player = pd.read_sql_query(query, connection)
game_num_by_player

,playerid,game_num
0,aaronha01,25
1,musiast01,24
2,mayswi01,24
3,mantlmi01,20
4,willite01,19
...,...,...
1671,chrisru01,0
1672,barrere01,0
1673,wyseha01,0
1674,grimeos01,0


## 4. Which school has generated the most distinct players?

In [ ]:
schools_playing = '/Users/greenapple/Downloads/lahman-csv_2014-02-14/SchoolsPlayers.csv'
schools_playing_df = pd.read_csv(schools_playing)
schools_playing_df.to_sql('schools_playing', con=engine, if_exists='replace', index_label=True)

In [125]:
engine.table_names()

['allstarfull', 'salaries', 'schools', 'college_playing', 'schools_playing']

In [42]:
query = '''
SELECT "schoolID", COUNT(DISTINCT "playerID") AS dist_players
FROM schools_playing
GROUP BY "schoolID"
ORDER BY dist_players DESC;
'''
school_dist_player = pd.read_sql_query(query, connection)
school_dist_player

,schoolID,dist_players
0,usc,102
1,texas,100
2,arizonast,98
3,stanford,82
4,michigan,77
...,...,...
708,wisuperior,1
709,wlmcarey,1
710,wlmjewell,1
711,woosteroh,1


## 5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`]

In [43]:
master = '/Users/greenapple/Downloads/lahman-csv_2014-02-14/Master.csv'
master_df = pd.read_csv(master)
master_df.to_sql('master', con=engine)

In [89]:
query = '''
WITH years AS
(SELECT playerid, MIN(yearid) AS first_year, MAX(yearid) AS last_year
FROM salaries
GROUP BY playerid)
SELECT last_year - first_year AS career_length 
FROM years
ORDER BY career_length DESC;
'''
player_1_and_last_years = pd.read_sql_query(query, connection)
player_1_and_last_years

,career_length
0,26
1,25
2,23
3,22
4,22
...,...
4663,0
4664,0
4665,0
4666,0


## 6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

In [45]:
query = '''
SELECT *
FROM master;
'''
master = pd.read_sql_query(query, connection)
master

,index,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,...,Aardsma,David Allan,205.0,75.0,R,R,2004-04-06,2013-09-28,aardd001,aardsda01
1,1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18349,18349,zupcibo01,1966.0,8.0,18.0,USA,PA,Pittsburgh,NaN,NaN,...,Zupcic,Robert,220.0,76.0,R,R,1991-09-07,1994-08-04,zupcb001,zupcibo01
18350,18350,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,...,Zupo,Frank Joseph,182.0,71.0,L,R,1957-07-01,1961-05-09,zupof101,zupofr01
18351,18351,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,...,Zuvella,Paul,173.0,72.0,R,R,1982-09-04,1991-05-02,zuvep001,zuvelpa01
18352,18352,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,NaN,NaN,...,Zuverink,George,195.0,76.0,R,R,1951-04-21,1959-06-15,zuveg101,zuverge01


In [83]:
query = '''
SELECT EXTRACT(month FROM DATE(debut)) as month, COUNT(EXTRACT(month FROM DATE(debut))) AS month_count
FROM master
GROUP BY month
ORDER BY month_count DESC;
'''
debut = pd.read_sql_query(query, connection)
debut

,month,month_count
0,9.0,5061
1,4.0,4711
2,5.0,2230
3,7.0,1978
4,8.0,1943
5,6.0,1893
6,10.0,308
7,3.0,41
8,NaN,0


## 7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [60]:
query1 = '''
SELECT playerid, AVG(salary) AS mean_salary
FROM salaries
LEFT JOIN master
ON salaries.playerid = master."playerID"
GROUP BY playerid
ORDER BY mean_salary DESC;
'''
join1_salary = pd.read_sql_query(query1, connection)
join1_salary

,playerid,mean_salary
0,rodrial01,1.777081e+07
1,jeterde01,1.403434e+07
2,teixema01,1.322955e+07
3,howarry01,1.315688e+07
4,sabatcc01,1.230357e+07
...,...,...
4663,mizerjo01,6.000000e+04
4664,soffra01,6.000000e+04
4665,romanjo02,6.000000e+04
4666,johnsjo04,6.000000e+04


In [61]:
query2 = '''
SELECT playerid, AVG(salary) AS mean_salary
FROM master 
LEFT JOIN salaries
ON salaries.playerid = master."playerID"
GROUP BY playerid
ORDER BY mean_salary DESC;
'''
join2_salary = pd.read_sql_query(query2, connection)
join2_salary

,playerid,mean_salary
0,None,NaN
1,rodrial01,1.777081e+07
2,jeterde01,1.403434e+07
3,teixema01,1.322955e+07
4,howarry01,1.315688e+07
...,...,...
4664,clibust02,6.000000e+04
4665,tomlida01,6.000000e+04
4666,tillmru01,6.000000e+04
4667,martimi01,6.000000e+04


In [ ]:
# Master table has a row with no salary (NaN). Salaries does not have any empty salary rows.